In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import nept
from core import Rat, combine_rats
from load_data import assign_label
from plotting import plot_behavior, plot_duration

In [ ]:
data_filepath = os.path.join(os.getcwd(), 'cache/data/spring2017')
output_filepath = os.path.join(os.getcwd(), 'plots/spring2017')

In [ ]:
sessions = ['!2017-04-17', '!2017-04-18', '!2017-05-17']

In [ ]:
# rats = ['R141', 'R142', 'R143', 'R144', 'R145', 'R146', 'R147', 'R148']
rats = ['R145']
groups = [1, 2, 2, 1, 2, 1, 1, 2]
males = ['R141', 'R143', 'R145', 'R147']
females = ['R142', 'R144', 'R146', 'R148']
group1 = ['R141', 'R144', 'R146', 'R147']
group2 = ['R142', 'R143', 'R145', 'R148']

cue_duration = 10
iti_buffer = 2

data = dict()
for rat in rats:
    data[rat] = Rat(rat, group1, group2)

for session in sessions:
    rats_data = nept.load_medpc(os.path.join(data_filepath, session), assign_label)

    for rat in rats:
        iti_starts = []
        iti_stops = []
        for trial in ['trial1', 'trial2', 'trial3', 'trial4']:
            iti_starts.extend(rats_data[rat][trial].starts - cue_duration - iti_buffer)
            iti_stops.extend(rats_data[rat][trial].starts - iti_buffer)
        rats_data[rat]['iti'] = nept.Epoch(np.vstack([iti_starts, iti_stops]))
        
        post_rewarded_starts = []
        post_rewarded_stops = []
        for trial in ['trial2', 'trial4']:
            post_rewarded_starts.extend(rats_data[rat][trial].stops)
            post_rewarded_stops.extend(rats_data[rat][trial].stops + cue_duration)
        rats_data[rat]['post_rewarded'] = nept.Epoch(np.vstack([post_rewarded_starts, post_rewarded_stops]))

        post_unrewarded_starts = []
        post_unrewarded_stops = []
        for trial in ['trial1', 'trial3']:
            post_unrewarded_starts.extend(rats_data[rat][trial].stops)
            post_unrewarded_stops.extend(rats_data[rat][trial].stops + cue_duration)
        rats_data[rat]['post_unrewarded'] = nept.Epoch(np.vstack([post_unrewarded_starts, post_unrewarded_stops]))

    for rat, group in zip(rats, groups):
        data[rat].add_session(**rats_data[rat], group=group)

n_sessions = len(data[rats[0]].sessions)

df = combine_rats(data, rats, n_sessions)

In [ ]:
for rat in rats:
    filename = rat + '_outcome_behavior.png'
    filepath = os.path.join(output_filepath, filename)
    plot_behavior(df, [rat], filepath=None, by_outcome=True)